# Problem Noting:
## Camera Depth
Closer Person and Farther Person from the camera lead to different skeletons coordinate making the action detection model prone to changes in 3D environment. Since joint locations are detected in pixel coordinates, a person who is far away will have joint coordinates that appear compressed, while those who are closer will appear expanded.  
-> Solution: Normalize Person Coordinate and calculate joint distances.
+ Joint locations are Normalized using equation (1) where $(x_i, y_i)$ and $(x'_i, y'_i)$ are the original joint coordinate and normalized joint coordinate in i-th position. Thus the normalized joints reprepresent n features

# 0. Install and Import Dependencies

In [1]:
import os
import time
import cv2 as cv
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt

# 1. Loading a Video and Saving Frames

In [2]:
import cv2 as cv
import os

def video_to_frames(video_path, output_folder, skip_rate=5):
    """
    Extract frames from a video and save them as .jpg files.
    skip_rate: Save 1 frame every 'skip_rate' frames to reduce data size if needed.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv.VideoCapture(video_path)
    frame_count = 0
    saved_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Save every 'skip_rate' frames (optional)
        if frame_count % skip_rate == 0:
            frame_filename = f"frame_{frame_count}.jpg"
            frame_path = os.path.join(output_folder, frame_filename)
            cv.imwrite(frame_path, frame)
            saved_count += 1
        
        frame_count += 1

    cap.release()
    print(f"Saved {saved_count} frames from {video_path} to {output_folder}")

### Loop through all video to extract frames

In [3]:
frame_output_root = "frames" # where extracted frames go

import glob

base_dir = "Single_person_violent"
classes = ["Kicking", "Punching", "Non-violent"]

X_sequences = []  # list of arrays, each array is shape (T, 132)
y_labels = []     # matching list of string labels

if not os.path.exists(frame_output_root):
    print(f"Creating directory '{frame_output_root}' for extracted frames.")

for cls in classes:
    class_video_dir = os.path.join(base_dir, cls)
    for video_file in os.listdir(class_video_dir):
        if video_file.endswith(".mp4") or video_file.endswith(".avi"):
            video_path = os.path.join(class_video_dir, video_file)
            # Output folder for frames of THIS video
            output_folder = os.path.join(frame_output_root, cls, video_file.split('.')[0])
            video_to_frames(video_path, output_folder, skip_rate=5)

else:
    print(f"The directory '{frame_output_root}' already exists. Skipping frame extraction.")

Creating directory 'frames' for extracted frames.
Saved 9 frames from Single_person_violent\Kicking\kicking1.mp4 to frames\Kicking\kicking1
Saved 15 frames from Single_person_violent\Kicking\kicking10.mp4 to frames\Kicking\kicking10
Saved 12 frames from Single_person_violent\Kicking\kicking11.mp4 to frames\Kicking\kicking11
Saved 16 frames from Single_person_violent\Kicking\kicking12.mp4 to frames\Kicking\kicking12
Saved 5 frames from Single_person_violent\Kicking\kicking13.mp4 to frames\Kicking\kicking13
Saved 7 frames from Single_person_violent\Kicking\kicking14.mp4 to frames\Kicking\kicking14
Saved 8 frames from Single_person_violent\Kicking\kicking15.mp4 to frames\Kicking\kicking15
Saved 6 frames from Single_person_violent\Kicking\kicking16.mp4 to frames\Kicking\kicking16
Saved 8 frames from Single_person_violent\Kicking\kicking17.mp4 to frames\Kicking\kicking17
Saved 17 frames from Single_person_violent\Kicking\kicking18.mp4 to frames\Kicking\kicking18
Saved 15 frames from Single_

### Resize Image for Training & Detection

In [62]:
import os
import cv2 as cv

def resize_images(image_folder, max_width=640, max_height=640):
    """
    Resizes all images in the given folder to the specified width and height,
    maintaining aspect ratio.
    """
    # Get a list of all files in the image folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        
        # Read the image using OpenCV
        img = cv.imread(image_path)
        
        # Check if the image was successfully read
        if img is None:
            print(f"Could not read image: {image_path}")
            continue
        
        height, width = img.shape[:2]

        # Calculate the scaling factor
        width_scale = max_width / width
        height_scale = max_height / height
        scale = min(width_scale, height_scale)  # Choose the smaller scale to fit within both dimensions

        # Calculate the new dimensions
        new_width = int(width * scale)
        new_height = int(height * scale)
        
        # Resize the image
        resized_img = cv.resize(img, (new_width, new_height), interpolation=cv.INTER_AREA)
        
        # Save the resized image, overwriting the original
        cv.imwrite(image_path, resized_img)
        
        print(f"Resized {image_file} to {new_width}x{new_height}")

# Example usage:
# resize_images('image_frames')

## 2. Pose Detection & Export Estimation Data to CSV

### Set Up tools and import libs

In [63]:
import mediapipe as mp
import numpy as np
import csv

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

csv_filename = 'pose_landmarks.csv'
num_landmarks = 33  # MediaPipe Pose has 33 landmarks


### 2.2 Write CSV Header 

In [64]:
header = ['class_label']  # e.g. Kick, Punching, Non-violent
for i in range(num_landmarks):
    header += [f'x{i}', f'y{i}', f'z{i}', f'v{i}']

# Initialize CSV file
with open(csv_filename, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

### 2.3 Extract Landmarks Function

Each frame has 33 landmarks (MediaPipe Pose detects 33 distinct body keypoints). For each landmark, you get 4 values:

+ x (normalized horizontal coordinate)

+ y (normalized vertical coordinate)

+ z (relative depth)

+ visibility (confidence of that landmark being visible)

So for each frame: 
+ 33 landmarks × 4 values each = 132 features total.

In [65]:
def extract_pose_landmarks(image_path, label, pose_estimator, csv_file):
    frame = cv.imread(image_path)
    if frame is None:
        print(f"Could not read {image_path}")
        return
    
    # Convert BGR to RGB for MediaPipe
    rgb_image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    rgb_image.flags.writeable = False

    # Process
    results = pose_estimator.process(rgb_image)

    if not results.pose_landmarks:
        return  # No detection, skip

    # Flatten the 33 landmarks
    pose_row = []
    for lm in results.pose_landmarks.landmark:
        pose_row.extend([lm.x, lm.y, lm.z, lm.visibility])
    
    row = [label] + pose_row

    # Write to CSV
    with open(csv_file, 'a', newline='') as f_out:
        writer = csv.writer(f_out)
        writer.writerow(row)

### 3.4 Main Loop to Process All Frames using Mediapipe

In [66]:
import os
import cv2 as cv

# Use a "with" statement to keep the Pose model open
with mp_pose.Pose(min_detection_confidence=0.5, 
                  min_tracking_confidence=0.5) as pose_estimator:
    
    # We revisit the 'frames' folder structure
    for cls in classes:  # Kick, Punching, Non-violent
        class_frame_dir = os.path.join("frames", cls)
        
        # subfolders: e.g. "kicking1", "kicking2"
        for video_folder in os.listdir(class_frame_dir):
            video_path = os.path.join(class_frame_dir, video_folder)
            if not os.path.isdir(video_path):
                continue
            
            # each file is a frame
            frame_files = [f for f in os.listdir(video_path) 
                           if f.lower().endswith(('.jpg', '.png'))]
            frame_files.sort()
            
            for frame_file in frame_files:
                frame_path = os.path.join(video_path, frame_file)
                extract_pose_landmarks(frame_path, cls, pose_estimator, csv_filename)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'frames\\Kick'

When this loop finishes, pose_landmarks.csv will have one row per frame, with 133 columns: `[class_label, x0, y0, z0, v0, x1, y1, z1, v1, ..., x32, y32, z32, v32]`.

class_label is “Kick,” “Punching,” or “Non-violent,” depending on the folder.

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

## 4. Dataset Evaluattion and Split Dataset for Train and Validation 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('pose_landmarks.csv')
df.describe().T

,count,mean,std,min,25%,50%,75%,max
x0,1557.0,0.270993,0.037887,0.160436,0.248814,0.274044,0.293484,0.474454
y0,1557.0,0.432748,0.147763,-0.112364,0.330139,0.455499,0.542123,0.862946
z0,1557.0,-0.201858,0.156447,-1.084517,-0.301598,-0.203921,-0.084499,0.207913
v0,1557.0,0.999301,0.003524,0.946393,0.999680,0.999839,0.999926,0.999993
x1,1557.0,0.259424,0.039268,0.145300,0.236737,0.263169,0.282650,0.468079
...,...,...,...,...,...,...,...,...
v31,1557.0,0.975100,0.018599,0.878272,0.969516,0.979351,0.987397,0.998316
x32,1557.0,0.659387,0.087442,0.232325,0.623651,0.671830,0.716763,0.974245
y32,1557.0,0.476513,0.196191,-0.077391,0.331915,0.467582,0.584179,1.027102
z32,1557.0,0.366132,0.228469,-0.770606,0.227527,0.380349,0.519203,0.882853


In [ ]:
df[df['class_label']=='Kicking'].T.head()

,0,1,2,3,4,5,6,7,8,9,...,726,727,728,729,730,731,732,733,734,735
class_label,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,...,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking,Kicking
x0,0.303104,0.332366,0.341824,0.336903,0.342331,0.33973,0.313527,0.318151,0.316202,0.316802,...,0.270538,0.264021,0.316085,0.271231,0.265952,0.263943,0.264104,0.264316,0.264757,0.317434
y0,0.329654,0.329657,0.317274,0.3075,0.281803,0.26157,0.304855,0.256063,0.250517,0.255854,...,0.574783,0.558444,0.596108,0.542123,0.520092,0.499933,0.490747,0.48139,0.478819,0.599363
z0,-0.425171,-0.387996,-0.464608,-0.547959,-0.529404,-0.513624,-0.378542,-0.500729,-0.499527,-0.352072,...,-0.265361,-0.340383,-0.111857,-0.290745,-0.310434,-0.316804,-0.297326,-0.29469,-0.253086,-0.087718
v0,0.99991,0.999917,0.99992,0.999879,0.999885,0.999894,0.999904,0.999913,0.99992,0.999922,...,0.999346,0.999396,0.999455,0.999496,0.999539,0.999582,0.999621,0.999656,0.999678,0.999709


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv('pose_landmarks.csv')

# Separate out the label
y_str = df['class_label'].values  # e.g. "Kick", "Punching", "Non-violent"

# Convert to numeric 0, 1, 2
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_str) 
# Suppose: "Kick" -> 0, "Punching" -> 2, "Non-violent" -> 1 (the exact mapping depends on alphabetical order or input)

X = df.drop('class_label', axis=1).values  # shape: (num_frames, 132)

# Optional: scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, 
    y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # keep class balance
)

print("Data shapes:", X_train.shape, X_test.shape)
print("Label distribution (train):", pd.Series(y_train).value_counts())
print("Label distribution (test):", pd.Series(y_test).value_counts())

Data shapes: (1245, 132) (312, 132)
Label distribution (train): 0    588
2    585
1     72
Name: count, dtype: int64
Label distribution (test): 0    148
2    146
1     18
Name: count, dtype: int64


## 5. Train a Classifier on Pose CSV

### 5.1 Train Machine Learning Classification Model

In [39]:
from sklearn.pipeline import make_pipeline 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [41]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [42]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [43]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [44]:
fit_models['rc'].predict(X_test)

array([2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 0, 2, 2, 0, 0, 0, 1, 2, 0, 0,
       2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 2,
       1, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 2, 1, 2, 2, 0, 0,
       2, 0, 0, 0, 2, 2, 1, 0, 0, 2, 1, 0, 0, 0, 2, 2, 2, 0, 0, 2, 2, 0,
       0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0,
       0, 0, 1, 0, 2, 2, 2, 2, 1, 2, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0,
       0, 2, 0, 0, 0, 2, 0, 2, 0, 1, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 0, 0,
       0, 2, 0, 0, 2, 2, 0, 2, 1, 2, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 0, 1, 2, 2, 0, 2, 0, 2, 0, 2, 0, 0,
       0, 2, 1, 1, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 0, 2,
       2, 0, 2, 2, 2, 0, 0, 1, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 1,
       0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 0, 2,
       0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 2,
       2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 2, 2, 0, 0, 2,

## 4.3 Train TensorFlow Model (Multi-Class)

## 4.3.1 DNN 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

num_classes = len(label_encoder.classes_)  # should be 3

lr_model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # 132
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')  # for 3-class classification
])

lr_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

lr_model.summary()

history = lr_model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         8,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,691 (41.76 KB)

 Trainable params: 10,691 (41.76 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5153 - loss: 1.0754 - val_accuracy: 0.8032 - val_loss: 0.4377
Epoch 2/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7946 - loss: 0.5088 - val_accuracy: 0.9116 - val_loss: 0.2931
Epoch 3/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8313 - loss: 0.4316 - val_accuracy: 0.9317 - val_loss: 0.2458
Epoch 4/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8757 - loss: 0.3232 - val_accuracy: 0.9438 - val_loss: 0.2007
Epoch 5/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8930 - loss: 0.2807 - val_accuracy: 0.9438 - val_loss: 0.1793
Epoch 6/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8937 - loss: 0.2649 - val_accuracy: 0.9518 - val_loss: 0.1470
Epoch 7/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9315 - loss: 0.1911 - val_accuracy: 0.9518 - val_loss: 0.1299
Epoch 8/20
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9218 - loss: 0.1977 - val_accuracy: 0.9558 - val_loss

In [48]:
test_loss, test_acc = lr_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Optional confusion matrix
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = np.argmax(lr_model.predict(X_test), axis=1)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Show detailed metrics
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9800 - loss: 0.0509 
Test Accuracy: 98.40%
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Confusion Matrix:
 [[145   0   3]
 [  0  18   0]
 [  2   0 144]]
              precision    recall  f1-score   support

     Kicking       0.99      0.98      0.98       148
 Non-violent       1.00      1.00      1.00        18
    Punching       0.98      0.99      0.98       146

    accuracy                           0.98       312
   macro avg       0.99      0.99      0.99       312
weighted avg       0.98      0.98      0.98       312



In [50]:
import pickle 

with open('lr_behaviour.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

### 4.3.2. LSTM 

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

num_classes = len(label_encoder.classes_)  # should be 3

model = tf.keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),  # 132
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')  # for 3-class classification
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32
)


# 5. Make Detection with Model

In [ ]:
with open('lr_behaviour.pkl', 'rb') as f:
    model = pickle.load(f)

In [51]:
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

In [52]:
cap = cv.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)
        
        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv.putText(image, body_language_class, coords, 
                        cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            
            # Get status box
            cv.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv.putText(image, 'CLASS'
                        , (95,12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
            cv.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            
            # Display Probability
            cv.putText(image, 'PROB'
                        , (15,12), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv.LINE_AA)
            cv.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv.LINE_AA)
            
        except:
            pass
                        
        cv.imshow('Raw Webcam Feed', image)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv.destroyAllWindows()